In [29]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import numpy as np
import pypyodbc
import csv
import os
import tempfile
import statsmodels.formula.api as smf
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [30]:
puma_ref = pd.read_excel('http://www.nyc.gov/html/dcp/download/census/nyc2010census_tabulation_equiv.xlsx',header=3)
# alternatively, use shapefile from https://data.cityofnewyork.us/Housing-Development/Public-Use-Microdata-Areas-PUMA-/cwiz-gcty
puma_list = puma_ref['PUMA'].unique()[~np.isnan(puma_ref['PUMA'].unique())]

In [31]:
pums_data = pd.read_csv('ss14hny.csv')
pums_data['PUMA'] = pums_data[['PUMA00','PUMA10']].max(axis=1)

In [32]:
nyc_pums = pd.DataFrame()
for puma in puma_list:
    nyc_pums = nyc_pums.append(pums_data.query('PUMA == @puma'))
    print nyc_pums.shape[0],'rows after addition of PUMA',str(puma)

2532 rows after addition of PUMA 3704.0
4290 rows after addition of PUMA 3706.0
7271 rows after addition of PUMA 3705.0
9141 rows after addition of PUMA 3703.0
11081 rows after addition of PUMA 3708.0
13603 rows after addition of PUMA 3702.0
15801 rows after addition of PUMA 3707.0
18926 rows after addition of PUMA 3710.0
21257 rows after addition of PUMA 3701.0
24034 rows after addition of PUMA 3709.0
27707 rows after addition of PUMA 4017.0
30805 rows after addition of PUMA 4013.0
33408 rows after addition of PUMA 4003.0
36267 rows after addition of PUMA 4014.0
38410 rows after addition of PUMA 4018.0
41972 rows after addition of PUMA 4004.0
44269 rows after addition of PUMA 4007.0
46446 rows after addition of PUMA 4002.0
50660 rows after addition of PUMA 4009.0
53223 rows after addition of PUMA 4005.0
55807 rows after addition of PUMA 4006.0
57735 rows after addition of PUMA 4011.0
60478 rows after addition of PUMA 4008.0
63099 rows after addition of PUMA 4010.0
66109 rows after add

In [33]:
AMI_60_max = {1:36300,2:41460,3:46620,4:51780,5:56940,6:62100,7:67260,8:72420}
# 80 pct figures for 2015 from http://www1.nyc.gov/assets/hpd/downloads/pdf/NSP-tenant-lease-guidelines.pdf
AMI_80_max = {1:48400,2:55280,3:62160,4:69040,5:74640,6:80160,7:85680,8:91200}
nyc_pums['AMI_60'] = 0
for n in AMI_60_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_60'] = 1
nyc_pums['AMI_80'] = 0
for n in AMI_80_max.items():
    nyc_pums.loc[(nyc_pums['NP']==n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1
nyc_pums.loc[(nyc_pums['NP']>n[0]) & (nyc_pums['HINCP']<=n[1]),'AMI_80'] = 1

In [34]:
age_grouping = {1:'PreWar',2:'PostWar',3:'PostWar',4:'PostWar',5:'PostWar',6:'PostWar',7:'PostWar',8:'Modern'}
nyc_pums['AgeGroup'] = ''
for n in age_grouping.items():
    nyc_pums.loc[nyc_pums['YBL']==n[0],'AgeGroup'] = n[1]
nyc_pums.loc[nyc_pums['YBL']>n[0],'AgeGroup'] = n[1]

In [35]:
size_grouping = {1:'Small',2:'Small',3:'Small',4:'Small',5:'Medium',6:'Medium',7:'Medium',8:'Large',9:'Large'}
nyc_pums['SizeGroup'] = ''
for n in size_grouping.items():
    nyc_pums.loc[nyc_pums['BLD']==n[0],'SizeGroup'] = n[1]
nyc_pums.loc[nyc_pums['BLD']>n[0],'SizeGroup'] = n[1]

In [36]:
nyc_pums['ResidencyGroup'] = ''
nyc_pums.loc[nyc_pums['MV']<=4,'ResidencyGroup'] = 'Under10'
nyc_pums.loc[nyc_pums['MV']==5,'ResidencyGroup'] = '10Years'
nyc_pums.loc[nyc_pums['MV']==6,'ResidencyGroup'] = '20Years'
nyc_pums.loc[nyc_pums['MV']==7,'ResidencyGroup'] = '30Years'

In [37]:
print ('Median HH income in all NYC PUMAs')
nyc_pums['HINCP'].median()

Median HH income in all NYC PUMAs


52000.0

In [38]:
print('Example median HH income in Williamsburg/Greenpoint')
nyc_pums[nyc_pums['PUMA10']==4001]['HINCP'].median()

Example median HH income in Williamsburg/Greenpoint


60000.0

In [39]:
# excluding group quarters and vacant household records
nyc_pums = nyc_pums.query('TYPE == 1 & BLD > 1 & BLD < 10')
nyc_pums = nyc_pums[nyc_pums['VACS'].isnull()]

In [40]:
print 'Percentage of households who own that are considered Low Income:', nyc_pums.query('TEN < 3')['AMI_80'].mean()
nyc_pums.query('TEN < 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who own that are considered Low Income: 0.364142139268


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.199559,0.354994,0.229702
Medium,0.321600,0.432099,0.369237
Small,0.309593,0.378939,0.385124


In [41]:
print 'Percentage of households who rent that are considered Low Income:', nyc_pums.query('TEN == 3')['AMI_80'].mean()
nyc_pums.query('TEN == 3').pivot_table(index='SizeGroup',columns='AgeGroup',values='AMI_80',aggfunc='mean')

Percentage of households who rent that are considered Low Income: 0.650481622307


AgeGroup,Modern,PostWar,PreWar
SizeGroup,,,
Large,0.512022,0.678588,0.672247
Medium,0.625724,0.659014,0.612135
Small,0.671775,0.636128,0.635859


In [42]:
print 'Also add length of residency category...'
print 'Percentage of households who rent that are considered Low Income:', nyc_pums.query('TEN == 3')['AMI_80'].mean()
nyc_pums.query('TEN == 3').pivot_table(index='SizeGroup',columns=['ResidencyGroup','AgeGroup'],values='AMI_80',aggfunc='mean')

Also add length of residency category...
Percentage of households who rent that are considered Low Income: 0.650481622307


ResidencyGroup   10Years                       20Years             30Years  \
AgeGroup          Modern   PostWar    PreWar   PostWar    PreWar   PostWar   
SizeGroup                                                                    
Large           0.785714  0.745490  0.751764  0.742718  0.749677  0.757719   
Medium          0.823529  0.733671  0.666317  0.770408  0.709966  0.760870   
Small           0.696970  0.627837  0.668421  0.625616  0.661064  0.690141   

ResidencyGroup             Under10                      
AgeGroup          PreWar    Modern   PostWar    PreWar  
SizeGroup                                               
Large           0.732801  0.500962  0.610944  0.611317  
Medium          0.731029  0.615478  0.616780  0.568275  
Small           0.696379  0.670667  0.635314  0.617595

In [14]:
# can we identify renters who live in condo buildings?
nyc_pums.query('TEN == 3 and FCONP == 1')
# it appears not

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP77,WGTP78,WGTP79,WGTP80,PUMA,AMI_60,AMI_80,AgeGroup,SizeGroup,ResidencyGroup


In [15]:
# dependents to test: HINCP, RNTP, GRNTP, NPF, TAXP
# independents to test: YBL (ord), TEN (c), BDSP, BLD, VACS - perhaps as a filter?
# also of interest: MV

In [16]:
# predict rent paid for renters
# mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [17]:
# predict income for renters
# mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [18]:
# predict income for owners
# mod = smf.ols(formula='HINCP ~ BLD + BDSP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
# print(mod.summary())

In [19]:
# predict income for owners with property tax as independent, but without bedrooms or size of building (covairance)
# mod = smf.ols(formula='HINCP ~ TAXP + C(PUMA) + C(YBL)', data = nyc_pums[nyc_pums['TEN']<3]).fit()
# print(mod.summary())

In [20]:
# predict rent paid for renters with grouped building ages
# mod = smf.ols(formula='RNTP ~ BLD + BDSP + C(PUMA) + C(AgeGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
# print(mod.summary())

In [24]:
# predict income for renters with grouped building ages/sizes and length of residence
mod = smf.ols(formula='HINCP ~ NP + C(SizeGroup) + C(AgeGroup) + C(PUMA) + C(ResidencyGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                  HINCP   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     268.8
Date:                Sun, 28 Feb 2016   Prob (F-statistic):               0.00
Time:                        12:52:56   Log-Likelihood:            -9.9448e+05
No. Observations:               78900   AIC:                         1.989e+06
Df Residuals:                   78837   BIC:                         1.990e+06
Df Model:                          62                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
Intercept       

In [27]:
logit_mod = smf.logit(formula='AMI_80 ~ C(SizeGroup) + C(AgeGroup) + C(PUMA) + C(ResidencyGroup)', data = nyc_pums[nyc_pums['TEN']==3]).fit()
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.581092
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 AMI_80   No. Observations:                78900
Model:                          Logit   Df Residuals:                    78838
Method:                           MLE   Df Model:                           61
Date:                Sun, 28 Feb 2016   Pseudo R-squ.:                  0.1021
Time:                        12:58:49   Log-Likelihood:                -45848.
converged:                       True   LL-Null:                       -51060.
                                        LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
Intercept                        0.4430      0.072      6.148      0.000

In [28]:
mfx = logit_mod.get_margeff()
print mfx.summary()

        Logit Marginal Effects       
Dep. Variable:                 AMI_80
Method:                          dydx
At:                           overall
                                  dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
C(SizeGroup)[T.Medium]          -0.0199      0.004     -5.042      0.000        -0.028    -0.012
C(SizeGroup)[T.Small]           -0.0545      0.005    -10.879      0.000        -0.064    -0.045
C(AgeGroup)[T.PostWar]           0.0433      0.006      6.830      0.000         0.031     0.056
C(AgeGroup)[T.PreWar]            0.0354      0.006      5.474      0.000         0.023     0.048
C(PUMA)[T.3702]                  0.1585      0.018      8.755      0.000         0.123     0.194
C(PUMA)[T.3703]                  0.0259      0.020      1.289      0.197        -0.013     0.065
C(PUMA)[T.3704]                  0.1094      0.017      6.302      0.000